In [1]:
# import copy
# import json
# import queue
# import shutil
import torch
import torch.nn as nn
# from argparse import Namespace

# import os

/Mounts/rbg-storage1/users/itamarc/miniconda3/envs/playground/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
ckpt_path = "/Mounts/rbg-storage1/users/itamarc/SybilX/r8w-00001.pth.tar"

In [3]:
# from torchvision import models
from torchvision.models import resnet18

In [4]:
TASKS = "tasks"
CHEXPERT_TASKS = ["No Finding",
                  "Enlarged Cardiomediastinum",
                  "Cardiomegaly",
                  "Lung Lesion",
                  "Airspace Opacity",
                  "Edema",
                  "Consolidation",
                  "Pneumonia",
                  "Atelectasis",
                  "Pneumothorax",
                  "Pleural Effusion",
                  "Pleural Other",
                  "Fracture",
                  "Support Devices"
                  ]

In [5]:
ckpt_dict = torch.load(ckpt_path) #, map_location=device)

In [6]:
model = resnet18(CHEXPERT_TASKS)

In [7]:
state_dict = ckpt_dict['state_dict']

In [8]:
for k in list(state_dict.keys()):
    # retain only encoder_q up to before the embedding layer
    if k.startswith('module.encoder_q') and not k.startswith('module.encoder_q.fc'):
        # remove prefix
        state_dict[k[len("module.encoder_q."):]] = state_dict[k]
        # delete renamed or unused k
        del state_dict[k]
    elif 'encoder_k' in k or 'module.queue' in k:
        del state_dict[k]
    elif k.startswith('module.encoder_q.fc'):
        del state_dict[k]
# for k in list(state_dict.keys()):        
#     state_dict[k.replace('module.model.', '')] = state_dict[k]
#     del state_dict[k]

In [9]:
model.load_state_dict(state_dict, strict=False)

_IncompatibleKeys(missing_keys=['fc.weight', 'fc.bias'], unexpected_keys=[])

In [16]:
nn.Sequential(*list(model.children())[:-1])

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [ ]:
# class Args:
#     def __init__(self):
#         self.moco = True

In [7]:
# model_args = Args()

In [8]:
dataset_nlst_xrays = json.load(open('/Mounts/rbg-storage1/datasets/ACRIN_XRAY/xray_dataset.json', 'r'))

In [9]:
import numpy as np
import pydicom
import matplotlib.pyplot as plt
import albumentations as A

In [10]:
samples = np.random.choice(dataset_nlst_xrays, size=64)
batch = []
for s in samples:
#     display(samples[1]['accessions'][0]['image_series'])
    path = samples[1]['accessions'][0]['image_series'][0]['path']
    ds = pydicom.dcmread(path)
    img = ds.pixel_array.astype(np.int16)
    resizer = A.Resize(512, 512)
    img = torch.tensor(resizer(image=img)['image'])
    num_dims = len(img.shape)
    if num_dims == 2:
        img = img.unsqueeze(0)
    existing_chan = img.size()[0]
    if not existing_chan == 3:
        x = img.expand(3, *img.size()[1:])
    batch.append(x)
batch = torch.stack(batch)
print(batch.shape)

torch.Size([64, 3, 512, 512])


In [23]:
model2 = nn.Sequential(*list(list(model.children())[0].children())[:-1]).cpu()

In [24]:
model2(batch.float()).shape

torch.Size([64, 512, 1, 1])